In [1]:
!pip install transformers
!pip install sacrebleu sentencepiece
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 5.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import re
import glob
import pandas as pd

from huggingface_hub import notebook_login
from transformers import pipeline

In [4]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Traduzione

Configuro modelli HuggingFace per la traduzione da Inglese a Italiano

In [5]:
# Modello Traduzione Inglese-Italiano
en_model_checkpoint = "Helsinki-NLP/opus-mt-it-es"
translator_en = pipeline("translation", model=en_model_checkpoint)

/usr/local/lib/python3.8/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


### Configuro Classe per la traduzione

In [6]:
class Translate:
  def __init__(self,
               orig_filenamepath, 
               trad_filenamepath, 
               translator_en):
    
    self.orig_filenamepath = orig_filenamepath
    self.trad_filenamepath = trad_filenamepath

    self.translator_en = translator_en

  def text_cleaning(self, text):
    text = re.sub('\([^)]*\)', '', text)

    return text

  def translate(self):
    df_trad = pd.DataFrame(columns=['Original', 'Translate'])

    df_orig = pd.read_csv(self.orig_filenamepath)

    trad_en_1 = self.translator_en([self.text_cleaning(text) for text in df_orig['Translate']])

    df_trad['Original'] = df_orig['Original']
    df_trad['Translate'] = [text['translation_text'] for text in trad_en_1[:]]

    df_trad.to_csv(self.trad_filenamepath, index=False, encoding='utf-8')

### Traduzione

In [7]:
# PARAMETRI GLOBALI
root_folder = 'drive/MyDrive/Traduzione_Dantesca/Data_Translate/'

# DATI
data_folder_name_originale = 'data'
filename_originale_1 = 'inferno_v2.csv'
filename_originale_2 = 'inferno_gpt.csv'

data_folder_name_traduzione = 'data_out'
filenamepath_traduzione_1 = 'inferno_it_es.csv'
filenamepath_traduzione_2 = 'inferno_gpt_it_es.csv'

DATA_PATH_ORIGINAL = os.path.abspath(os.path.join(root_folder, data_folder_name_originale))
orig_filenamepath_1 = os.path.abspath(os.path.join(DATA_PATH_ORIGINAL, filename_originale_1))
orig_filenamepath_2 = os.path.abspath(os.path.join(DATA_PATH_ORIGINAL, filename_originale_2))

DATA_PATH_TRANSLATE = os.path.abspath(os.path.join(root_folder, data_folder_name_traduzione))
trad_filenamepath_1 = os.path.abspath(os.path.join(DATA_PATH_TRANSLATE, filenamepath_traduzione_1))
trad_filenamepath_2 = os.path.abspath(os.path.join(DATA_PATH_TRANSLATE, filenamepath_traduzione_2))

In [9]:
translate_inferno = Translate(orig_filenamepath=orig_filenamepath_1,
                              trad_filenamepath=trad_filenamepath_1,
                              translator_en=translator_en)

translate_inferno.translate()

In [8]:
translate_inferno = Translate(orig_filenamepath=orig_filenamepath_2,
                              trad_filenamepath=trad_filenamepath_2,
                              translator_en=translator_en)

translate_inferno.translate()